# Notebook #17: Study of the number of finite elements used for time span discretization

This Notebook is part of the Supporting Information for the paper *Cyclic steady state simulation and waveform design for Dynamic/Programmable Catalysis*, by C. C. Tedesco, J. R. Kitchin, and C. D. Laird [[1]](#1).

The purpose of this notebook is to simulate four dynamic catalysis systems through the simultaneous approach (framework using Pyomo.DAE [[2]](#2) and IPOPT [[3]](#3)), using a range of number for the number of finite elements (nfe), or intervals in which the time span is discretized in order to solve for the cyclic steady state. The discussion of these results is done in **Session S2** of the paper's Supporting Information.

Please see following code, comments and narrative text to understand and use the workflow.

## Install Pyomo and IPOPT on Google Colab

In [11]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin 
    os.environ['PATH'] += ':bin'

!pip install munch 

## Importing necessary libraries from Python and Pyomo

In [2]:
from munch import Munch
import numpy as np
from pyomo.environ import (ConcreteModel, Constraint, Objective, Var, Param,
                           SolverFactory, value, sin, cos, Set,
                           TransformationFactory, assert_optimal_termination, exp, RangeSet)
from pyomo.dae import ContinuousSet, DerivativeVar
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import sys
import pandas as pd
from scipy import signal
from tqdm import tqdm 
import time
import idaes
from scipy.optimize import minimize
import matplotlib.cm as cm
import matplotlib.colors as colors

## Declaring parameter values

In [3]:
class Params:
    """
    Declares reaction system and CSTR model parameters
    """
    
    def __init__(self,  T = 423.15, # K
                        q = 0.0008333, # L/s
                        Po = 1.01325, # bar
                        Rg_constants = 0.0083144626, # kJ/K-gmol
                        Rg_Caf = 0.083144626, # bar-L/K-gmol
                        Rg_ODEs = 0.083144626, # bar-L/K-gmol
                        kB = 1.380649e-23, # J/K
                        hp = 6.62607015e-34, # J-s
                        F = 96.485, # mC/gmol
                        alpha = 0.6, 
                        beta = 100,
                        gamma = 2, 
                        delta = 1.4, # eV
                        H1 = 0, # kJ/mol
                        H2 = 0, # kJ/mol
                        N = 2.76e-6, # gmol sites
                        V = 2.60e-4, # L
                        Caf = 2.8423101747069843, # mol/L
                        Cbf = 0, # mol/L
                        delS1 = -135, # J/mol-K
                        delS2 = 0, # J/mol-K
                        delS3 = 135, # J/mol-K
                        number_periods = 1,
                        nfe = 900
                        ):
        
        
        self.T = T # Temperature on the simulated CSTR
        self.Po = Po # Standard pressure
        self.Caf = Caf # Initial concentration of A
        self.Cbf = Cbf # Initial concentration of B
        self.Rg_constants  = Rg_constants # Gas constant used in kinetic constants calculation
        self.Rg_Caf = Rg_Caf # Gas constant used to calculate Caf
        self.Rg_ODEs = Rg_ODEs # Gas constant used in the ODEs
        self.kB = kB # Boltzmann constant
        self.hp = hp # Plank constant
        self.F = F  # Faraday constant, for unit conversion
        self.alpha = alpha # Parameter for the BEP relation, slope of line of the relashionship
        self.beta = beta # Parameter for the BEP relation, intercept of line of the relashionship
        self.gamma = gamma # Parameter for the BE relation, slope of line of the relashionship
        self.delta = delta # Parameter for the BE relation, common BE point
        self.H1 = H1 # Enthalpy of A
        self.H2 = H2 # Enthalpy of B
        self.V = V # Volume of the CSTR
        self.delS1 = delS1 # Entropy of A
        self.delS2 = delS2 # Entropy of reaction
        self.delS3 = delS3 # Entropy of B
        self.q = q # Volumetric rate of the reactor
        self.N = N # Number of active sites on the surface
        self.number_periods = number_periods # Number of periods to be simulated 
        self.nfe = nfe # Number of discretization points

## Calculate the periodic kinetic constants from the forcing signal for binding energies 

In [4]:
def time_dependent_params(params, t):
    """
    Calculates the periodic kinetic constants from the periodic binding energy of A
    
    Args:
        params: From where to take the parameter values. For now we only have one option, the class "Params"

    Returns:
        Munch callables for the kinetic constants
    """
    
    # Parameters as given in the class
    T = params.T
    Rg_constants = params.Rg_constants
    kB = params.kB
    hp = params.hp
    F = params.F
    alpha = params.alpha
    beta = params.beta
    gamma = params.gamma
    delta = params.delta
    H1 = params.H1
    H2 = params.H2
    delS1 = params.delS1
    delS2 = params.delS2
    delS3 = params.delS3
    amplitude = params.amplitude # Amplitude of the wave (decision variable)
    offset = params.offset # Offset of the wave: determines the values of binding energy of A (decision variable)
    freq_osci = params.freq_osci # Frequency of the wave (decision variable)
    duty_cycle = params.duty_cycle # Duty cycle of the square wave (decision variable)
    
    # Binding energy calculation (eV)
    # Oscillation in square waveform
    BEa = amplitude * signal.square(2 * np.pi * 1 * t, duty=duty_cycle) + offset
        
    # Binding energy values for B, from the linear relationship with Bea   
    BEb = gamma*BEa + H2/F - gamma*H1/F + (1 - gamma)*delta

    # Heat of reaction (kJ/mol)
    delH1 = -BEa*F # For the adsorption step
    delH2 = (H2 - BEb*F) - (H1 - BEa*F) # For the surface reaction step
    delH3 = BEb*F # For the desorption step

    # Activation energy from BEP relationship (kJ/mol)
    Ea = alpha*delH2 + beta

    # Gibbs free energy  
    delG1 = delH1 - T*delS1*0.001 # For the adsorption step
    delG2 = delH2 - T*delS2*0.001 # For the surface reaction step
    delG3 = delH3 - T*delS3*0.001 # For the desorption step

    # Equilibrium constants (unitless)
    K1 = np.exp(-delG1/Rg_constants/T) # For the adsorption step
    K2 = np.exp(-delG2/Rg_constants/T) # For the surface reaction step 
    K3 = np.exp(-delG3/Rg_constants/T) # For the desorption step

    # Kinetic constants (1/s)
    kf1 = ((kB*T)/hp)*np.exp(delS1*0.001/Rg_constants) # For the adsorption of A
    kf2 = ((kB*T)/hp)*np.exp(-Ea/Rg_constants/T) # For the forward surface reaction
    kr3 = ((kB*T)/hp)*np.exp(-delS3*0.001/Rg_constants) # For the adsorption of B
    kr1 = kf1/K1 # For the desorption of A
    kr2 = kf2/K2 # For the reverse surface reaction
    kf3 = kr3*K3 # For the desorption of B
    
    return Munch(kf1=kf1, kf2=kf2, kf3=kf3, kr1=kr1, kr2=kr2, kr3=kr3, BEa=BEa)

## Create the Pyomo model

In [5]:
def create_pyomo_model(params):
    """
    Create the pyomo model for the CSTR. Calls the discretization through Backwards Finite Difference 
    as a means to solve the ODE system modelled here. 
    
    Args:
        params: From where to take the parameter values. For now we only have one option, the class "Params"
        tfinal: period of the wave, calculated from the optimal frequency value

    Returns:
        Discretized Pyomo model
    """

    # Parameters as given in the class
    T = params.T
    q = params.q
    Po = params.Po
    Caf = params.Caf
    Cbf = params.Cbf
    Rg_constants = params.Rg_constants
    Rg_Caf = params.Rg_Caf
    Rg_ODEs = params.Rg_ODEs
    kB = params.kB
    hp = params.hp
    F = params.F
    alpha = params.alpha
    beta = params.beta
    gamma = params.gamma
    delta = params.delta
    H1 = params.H1
    H2 = params.H2
    amplitude = params.amplitude
    N = params.N
    V = params.V
    offset = params.offset
    delS1 = params.delS1
    delS2 = params.delS2
    delS3 = params.delS3
    freq_osci = params.freq_osci
    duty_cycle = params.duty_cycle
    number_periods = params.number_periods
    nfe = params.nfe
    
    
    # Declare pyomo model, parameters and time span
    m = ConcreteModel()
    m.params = params
    
    m.tau = ContinuousSet(bounds=(0, 1)) # Unscaled time
    m.times = Var(m.tau) # not a continuous set, but a variable that depends on tau
    m.tf = Param(initialize = params.number_periods*1/params.freq_osci)
    
    # Define variables- concentrations varying with time, 
    # initialized in values that would make sense to be the solution
    m.Ca = Var(m.tau, initialize=2.84)
    m.Cb = Var(m.tau, initialize=0.1)
    m.thetaA = Var(m.tau, initialize=0.5)
    m.thetaB = Var(m.tau, initialize=0.5)

    # Declare derivative variables
    m.dtimedtau = DerivativeVar(m.times)
    m.dCadt = DerivativeVar(m.Ca)
    m.dCbdt = DerivativeVar(m.Cb)
    m.dthetaAdt = DerivativeVar(m.thetaA)
    m.dthetaBdt = DerivativeVar(m.thetaB)

    
    @m.Constraint(m.tau)
    def _ode3(m,t):
        if t == 0:
            return Constraint.Skip
        return m.dtimedtau[t] == m.tf
    
    
    # Declare ODEs as contraints
    
    # Constraint that depends on time span
    @m.Constraint(m.tau)
    # Define equation that is a function of the model and t
    def dCadt_eq(m,t):
        # pt gets the result of the function 
        pt = time_dependent_params(m.params, t)
        # Declare theta_star since it varies with thetaA and B and therefore, with time
        theta_star = 1 - m.thetaA[t] - m.thetaB[t]
        # ODE, being that `munch` format was used to get the k's from the function
        return m.dCadt[t] == m.tf * ((q/V)*(Caf - m.Ca[t]) \
            - pt.kf1*m.Ca[t]*Rg_ODEs*T*(theta_star)/Po*(N/V) + pt.kr1*m.thetaA[t]*(N/V))

    @m.Constraint(m.tau)
    def dCbdt_eq(m,t):
        pt = time_dependent_params(m.params, t)
        theta_star = 1 - m.thetaA[t] - m.thetaB[t]
        return m.dCbdt[t] == m.tf * ((q/V)*(Cbf - m.Cb[t]) \
            + (pt.kf3*m.thetaB[t] - pt.kr3*m.Cb[t]*Rg_ODEs*T*(theta_star)/Po)*(N/V))

    @m.Constraint(m.tau)
    def dthetaAdt_eq(m,t):
        pt = time_dependent_params(m.params, t)
        theta_star = 1 - m.thetaA[t] - m.thetaB[t]
        return  m.dthetaAdt[t] == m.tf * (pt.kf1*m.Ca[t]*Rg_ODEs*T*(theta_star)/Po \
            - pt.kr1*m.thetaA[t] - pt.kf2*m.thetaA[t] + pt.kr2*m.thetaB[t])
    
    @m.Constraint(m.tau)
    def dthetaBdt_eq(m,t):
        pt = time_dependent_params(m.params, t)
        theta_star = 1 - m.thetaA[t] - m.thetaB[t]
        return m.dthetaBdt[t] == m.tf * ((pt.kf2*m.thetaA[t] - pt.kr2*m.thetaB[t]) \
            - (pt.kf3*m.thetaB[t] - pt.kr3*m.Cb[t]*Rg_ODEs*T*(theta_star)/Po))

    
    discretizer = TransformationFactory('dae.finite_difference')
    discretizer.apply_to(m,nfe=nfe,scheme='BACKWARD')
    
    # Return the model
    return m

## Solving the model and calculating avTOF from the results

In [6]:
def tof_from_decision_vars(params, tee=True):
    """
    Solves the Pyomo model and calculates avTOF from results of solving the BVP
    
    Args:
        params: From where to take the parameter values. For now we only have one option, the class "Params"
        tee = True: Returns the ipopt log, which is advised. Use Tee = False in case you do not want to see the log.

    Returns:
        Value of the time-averaged turnover frequency
    """
        
    # Get the Pyomo model with final time as the number of periods times the period of the forcing wave
    m = create_pyomo_model(params)
    
    # Key part of the approach!
    # Equality constraints for initial and final points - to guarantee periodicity 
    # and simulation of full cycles
    m.CaCSS = Constraint(expr= m.Ca[m.tau.first()] == m.Ca[m.tau.last()])
    m.CbCSS = Constraint(expr= m.Cb[m.tau.first()] == m.Cb[m.tau.last()])
    m.thetaACSS = Constraint(expr= m.thetaA[m.tau.first()] == m.thetaA[m.tau.last()])
    m.thetaBCSS = Constraint(expr= m.thetaB[m.tau.first()] == m.thetaB[m.tau.last()])

    # Call ipopt as solver on the square problem/ simulation
    solver = SolverFactory('ipopt')
    # If solving in a loop, terminate runs when unable to solve
    status = solver.solve(m, tee=tee)
    assert_optimal_termination(status)


    # Attributing arrays for calculating plotting
    Ca = [value(m.Ca[t]) for t in m.tau]
    Cb = np.asarray([value(m.Cb[t]) for t in m.tau])
    thetaA = [value(m.thetaA[t]) for t in m.tau]
    thetaB = [value(m.thetaB[t]) for t in m.tau]
    theta_star = [value(1 - m.thetaA[t] - m.thetaB[t]) for t in m.tau]

    # Calculating TOF from the concentration of product desorbed
    TOF = Cb*params.q/params.N
        
    # Calculating the time-average by integration with the trapezoid method
    avTOF = np.trapz(TOF/(1), m.tau) 
    
    return [TOF, avTOF]

## Establishing the parameters of the 4 case study systems and nfe range

In [7]:
params = Params()

offsets = [1.2, 1.35, 1.4, 1.1]
amplitudes = [0.3, 0.4, 0.35, 0.3]
frequencies = [1000, 1500, 700, 100]
duties = [0.5, 0.8, 0.3, 0.9]

nfes = np.arange(100, 1100, 100)

In [8]:
data = {
    'Frequency (Hz)': [f'{f:1.0f}' for f in frequencies],
    'Amplitude (eV)': amplitudes,
    'Offset (eV)': offsets,
    'Duty cycle': duties,
}

df = pd.DataFrame(data)

custom_names = ['System 1', 'System 2', 'System 3', 'System 4']
df.index = custom_names

print('Comparison of results\n')
print(df)

Comparison of results

         Frequency (Hz)  Amplitude (eV)  Offset (eV)  Duty cycle
System 1           1000            0.30         1.20         0.5
System 2           1500            0.40         1.35         0.8
System 3            700            0.35         1.40         0.3
System 4            100            0.30         1.10         0.9


In [9]:
avTOF_nfe = []
run_times = []

for params.offset, params.amplitude, params.freq_osci, params.duty_cycle in zip(offsets, amplitudes, frequencies, duties):

    for params.nfe in nfes:
        start_time = time.time()
        TOF, avTOF = tof_from_decision_vars(params, tee = False)
        end_time = time.time()
        elapsed_time = end_time - start_time
        avTOF_nfe.append(avTOF)
        run_times.append(elapsed_time)

In [10]:
# Separating into intervals to print a table for each system
intervals = [(0, 9), (10, 19), (20, 29), (30, 39)]

for i, interval in enumerate(intervals):
    start, end = interval
    data = {
        'Number of finite elements': nfes, 
        'avTOF (Hz)': [f'{tof:1.3f}' for tof in avTOF_nfe[start:end + 1]],
        'Run time (s)': [f'{run_time:1.3f}' for run_time in run_times[start:end + 1]]
    }
    df = pd.DataFrame(data)
    
    max_diff = np.max(avTOF_nfe[start:end + 1]) - np.min(avTOF_nfe[start:end + 1])
    max_diff_run = np.max(run_times[start:end + 1]) - np.min(run_times[start:end + 1])
    
    print(f"For System {i + 1}")
    print(df)
    print(f'Maximum difference between avTOF values = {max_diff:1.3f} s-1.')
    print(f'Maximum difference between run times values = {max_diff_run:1.3f} s-1.')
    print()

For System 1
   Number of finite elements avTOF (Hz) Run time (s)
0                        100     10.292        0.717
1                        200     10.292        1.647
2                        300     10.292        2.006
3                        400     10.292        3.125
4                        500     10.293        4.149
5                        600     10.293        4.483
6                        700     10.293        5.559
7                        800     10.293        6.296
8                        900     10.293        7.136
9                       1000     10.293        8.053
Maximum difference between avTOF values = 0.001 s-1.
Maximum difference between run times values = 7.335 s-1.

For System 2
   Number of finite elements avTOF (Hz) Run time (s)
0                        100    733.939        0.712
1                        200    735.017        1.913
2                        300    735.377        2.022
3                        400    735.558        3.180
4              

## Bibliography

<a id="1">1. link to our paper
    
<a id="2">2. Nicholson, J. D. Siirola, J. Watson, V. M. Zavala, and L. T. Biegler. pyomo.dae: a mod-
eling and automatic discretization framework for optimization with differential and algebraic
equations. Mathematical Programming Computation, 10(2):187–223, 2018.
    
<a id="3">3. A. Wachter and L. Biegler. On the implementation of an interior-point filter line-search
algorithm for large-scale nonlinear programming. Mathematical Programming, 106:25–57, 2006.